<center>
    <img src="https://github.com/juanandres-montero/macroeconometria/blob/main/logo%20EEC%20grande.png?raw=1" width="396" height="161"><br>
    <b style="font-size:200%">EC4301 MACROECONOMETRÍA</b><br>
    <b style="font-size:100%">Asistente: Juan Andrés Montero</b><br><br>
    <b style="font-size:100%"> Laboratorio #6:</b>
    <br>    
    <div style="font-size:150%; color:white; background-color: #0064b0; padding-top: 20px; padding-bottom: 20px; width: 50%; margin: 0 auto; border-radius: 15px;">
        ARIMA
    </div>
</center>

<br>

<i style="font-size:80%"> 
    Creado: Setiembre 2024
</i>

<hr>

# ARIMA

---

## Modelo ARIMA(p,d,q)

El modelo ARIMA combina tres componentes principales:
- **AR(p)**: Componente autorregresivo.
- **MA(q)**: Componente de media móvil.
- **d**: Veces que la serie es diferenciada para hacerla estatacionaria.

---

## Carga de Datos y Preparación

Vamos a utilizar los datos del **IPC** desde enero de 2008 hasta febrero de 2021. Primero, importamos la base de datos en Stata y generamos la variable de tiempo (mes).

```
clear all 
cd "C:/Downloads" 
use "Lab06.dta" 

gen mes = mofd(time)
format mes %tm
tsset mes
```

- `mofd(time)`: Convierte la variable de tiempo a formato mensual.
- `tsset`: Declara la variable de tiempo para análisis de series de tiempo.

---

## PASO 1: Identificación de los parámetros p, q, d

Para identificar los valores de $d$, $p$ y $q$, seguimos la **Metodología Box-Jenkins**.

### Identificación de \(d\): Prueba de estacionariedad

Usamos la prueba de **Dickey-Fuller Aumentada (DFA)** para verificar si la serie es estacionaria. Existen tres tipos de pruebas:



#### Serie con tendencia
```
dfuller cpi, trend regress
```
Se incluye una tendencia determinística en el modelo. Se agrega un término de tendencia lineal (un coeficiente multiplicado por el tiempo) en la regresión. Esto es útil cuando sospechas que la serie tiene una tendencia creciente o decreciente en el tiempo.
#### Serie con deriva
```
dfuller cpi, drift regress
```
Se incluye un término constante (también llamado "drift" o deriva), lo que significa que la serie puede tener una tendencia constante (es decir, una inclinación hacia arriba o hacia abajo sin ser necesariamente lineal como en el caso de una tendencia). No se incluye un término de tendencia lineal explícito.

#### Serie sin tendencia ni deriva
```
dfuller cpi, regress
```

En todos los casos, el resultado es que no se rechaza $H_0$, lo que indica que la serie **no es estacionaria**. 

### Diferenciación de la serie

Aplicamos la primera diferencia de la serie para hacerla estacionaria.

```
dfuller d.cpi, regress
```

Resultado: se rechaza $H_0$, lo que indica que la serie ahora **es estacionaria**. Conclusión: la serie es integrada de orden 1 $d=1$. 

---

## FAC y FACP
- **FAC** mide la correlación entre una variable y sus valores rezagados (anteriores) en diferentes intervalos de tiempo. Es decir, la FAC en el rezago 1 mide la correlación entre los valores de la serie en el tiempo $t$ y $t-1$; en el rezago 2, entre $t$ y $t-2$, y así sucesivamente.
    - Una autocorrelación cercana a 1 indica una fuerte correlación positiva entre los valores actuales y pasados, mientras que un valor cercano a -1 indica una fuerte correlación negativa.
 
- **FACP** mide la correlación entre una variable y sus valores rezagados, ajustando por las correlaciones con los rezagos intermedios. Es decir, muestra la correlación en el rezago $k$ eliminando el efecto de los rezagos intermedios (1, 2,..., $k-1$)
    - Si en el rezago 1 la FACP es significativa y luego cae rápidamente a 0, esto podría indicar que solo hay dependencia en el corto plazo (por ejemplo, un proceso AR(1))

## PASO 2: Determinación de ($p$) y ($q$) 

Usamos los **correlogramas** para determinar los valores de ($p$) y ($q$). 

### Determinación de ($q$)  (MA):
Usamos la **Función de Autocorrelación (FAC)**.

```
ac d.cpi
```
Un posible valor para ($q$)  es 1.


### Determinación de ($p$)  (AR): 
Usamos la **Función de Autocorrelación Parcial (FACP)**.

``` 
pac d.cpi
```
Posibles valores para ($p$): 1 o 2. 

Por lo tanto, los modelos candidatos son **ARIMA(1,1,1)** o **ARIMA(2,1,1)**. 

---

### Correlograma

Para visualizar ambas pruebas también se puede utilizar lo siguiente:

```corrgram d.cpi, lags(20)```

Que muestra el mismo resultado anterior.

## PASO 3: Estimación de parámetros

Estimamos los modelos ARIMA(1,1,1) y ARIMA(2,1,1) y comparamos sus **criterios de información**.

### ARIMA(1,1,1)
```stata
arima cpi, arima(1,1,1)
estat ic
```

### ARIMA(2,1,1)
```stata
arima cpi, arima(2,1,1)
estat ic
```

### Resultados:
- **Criterio de Akaike** favorece el modelo ARIMA(2,1,1).
- **Criterio de Bayes** favorece el modelo ARIMA(1,1,1).
- **Error cuadrático medio ($\sigma$)** es menor en el modelo ARIMA(2,1,1). 

Conclusión: se selecciona el modelo **ARIMA(2,1,1)**.

---

## PASO 4: Diagnóstico

### Verificación de ruido blanco en los errores

Calculamos los **residuos** del modelo ARIMA(2,1,1) y realizamos una prueba de ruido blanco (Ljung-Box).

```
arima cpi, arima(2,1,1)
predict error, resid
wntestq error
```

Resultado: no se rechaza $H_0$, lo que indica que los errores son ruido blanco.


### Verificación de estabilidad e invertibilidad

Usamos las **raíces del polinomio** para verificar la estabilidad del modelo.

```stata
estat aroots
```

Si todas las raíces están fuera del círculo unitario, el proceso es estable e invertible.

---

## Conclusión

El modelo final seleccionado es **ARIMA(2,1,1)**, y ha pasado todas las pruebas de diagnóstico.

---

# Autoarima

Se pueden encontrar los mejores modelos automáticamente 